In [1]:
#
# TODO: Agregar data básica del modelo
# * Agregar URLs a publicaciones, imágenes
# TODO: Hacer segundo research en base a los modelos a comparar
# * Los modelos del comparador algunos están descontinuados, se deberían de quitar
# ! Los modelos de la base del comparador no están completos, trae unos pocos
# ? SOP: https://docs.google.com/document/d/1zp91sPv0uYKKByQpszNrFQEFBcFzxP0HXX5vgh1azVw/edit?usp=sharing

In [2]:
from google import genai
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from config.paths import SRC_DIR
from db.loader import *
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Elegir país de los modelos

In [3]:
pais = "Colombia"

## Generar los datos básicos del modelo

### DB del comparador

In [4]:
df_comparador = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\src\data\db\bd_comparador_processed.csv")
df_comparador = df_comparador[df_comparador["stock"] == True]

### Base de inventario

In [5]:
db_loader = db_loader(pais)
db_loader.start()
df_columnas_finales = db_loader.data_base

c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\src\notebooks\..\db\loader.py:18: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data_base = pd.read_csv(os.getenv('CO_INVENTORY_PATH'))


In [24]:
df_columnas_finales[df_columnas_finales["code"] == "CO2961-hero-hunk-125-r"]

,Fecha de actualización,code,Marca,Modelo,Año,Tipo,Precio,url
99705,22/12/2025,CO2961-hero-hunk-125-r,Hero,Hunk 125 R,2026,['Urbana'],7490000,https://www.galgo.com/co/motos/CO2961-hero-hunk-125-r


Cambiar modelo

In [6]:
df_modelo = df_columnas_finales[df_columnas_finales["code"] == "CO2961-hero-hunk-125-r"]

In [7]:
for index, row in df_modelo.iterrows():
    marca = row["Marca"]
    modelo = row["Modelo"]
    año = row["Año"]
    pais = pais
    tipo = row["Tipo"]
    code = row["code"]

In [8]:
top_3_modelos = df_comparador[df_comparador["url"].str.contains(code)].head(3)
top_3_modelos

,url,Views,available_code,stock
4,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO731-tvs-raider-125,2291,"['CO2961-hero-hunk-125-r', 'CO731-tvs-raider-125']",True
19,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO2861-tvs-raider-fi-125,1272,"['CO2961-hero-hunk-125-r', 'CO2861-tvs-raider-fi-125']",True
152,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO2615-hero-hunk-150-xt,466,"['CO2961-hero-hunk-125-r', 'CO2615-hero-hunk-150-xt']",True


In [ ]:
import ast
import re
# Extraer los códigos de los modelos comparados (excluyendo el modelo principal)
modelos_comparados = []

for index, row in top_3_modelos.iterrows():
    available_codes = row["available_code"]
    # Si es una lista en string, conviértela a lista real
    if isinstance(available_codes, str):
        try:
            available_codes = ast.literal_eval(available_codes)
        except Exception:
            # En caso de fallo, intenta limpiar y separar
            available_codes = re.findall(r"(CO\d{3,}-[\w-]+)", available_codes) # ! ESTÁ HARDCODEANDO PAÍS
    # print("Códigos disponibles:", available_codes)

    # Limpiamos los códigos por si tienen espacios, corchetes, etc.
    available_codes = [c.strip(" []'\"") for c in available_codes]

    # Encontrar el código que NO es el modelo principal
    codigo_comparado = next((c for c in available_codes if c != code), None)
    # print("Código comparado:", codigo_comparado)

    # Buscar en df_columnas_finales para obtener Marca y Modelo
    modelo_info = df_columnas_finales[df_columnas_finales["code"] == codigo_comparado]

    if not modelo_info.empty:
        modelos_comparados.append({
            "marca": modelo_info.iloc[0]["Marca"],
            "modelo": modelo_info.iloc[0]["Modelo"]
        })

# Verificar que tenemos 3 modelos
# print(f"Modelos comparados encontrados: {len(modelos_comparados)}")
# for i, m in enumerate(modelos_comparados, 1):
#     print(f"{i}. {m['marca']} {m['modelo']}")

In [10]:
modelos_comparados

[{'marca': 'TVS', 'modelo': 'Raider 125'},
 {'marca': 'TVS', 'modelo': 'Raider FI 125'},
 {'marca': 'Hero', 'modelo': 'Hunk 150 XT'}]

### Prompt a enviar

In [11]:
from utils.prompt_utils import *

prompt_from_file = read_prompt_from_file(os.path.join(SRC_DIR, "data", "prompts", "knowledge_base_template.md"))
prompt = replace_variables(prompt_from_file, {
    "{marca}": marca,
    "{modelo}": modelo,
    "{año}": str(año),
    "{pais}": pais,
    "{tipo}": tipo
})

### Usando Gemini

In [12]:
client = genai.Client() # La API Key se lee automaticamente desde variables de entorno

# Llamada a la API
interaction_con = client.interactions.create(
    model='gemini-3-pro-preview',
    input=prompt,
    tools=[{'type': 'google_search'}]  # Habilita Google Search
)

C:\Users\JTRUJILLO\AppData\Local\Temp\ipykernel_10828\417860456.py:4: UserWarning: Interactions usage is experimental and may change in future versions.
  interaction_con = client.interactions.create(


### Output de la API

In [13]:
final_output = ""
for output in interaction_con.outputs:
    if hasattr(output, "text"):
        final_output += output.text

## Comparaciones de modelos

In [14]:
# Generar la lista de modelos comparados para el prompt
modelos_comparados_texto = ""
for i, m in enumerate(modelos_comparados, 1):
    modelos_comparados_texto += f"{i}. {m['marca']} {m['modelo']}\n"

# Leer el prompt de comparaciones
from utils.prompt_utils import *
prompt_comparaciones = read_prompt_from_file(
    os.path.join(SRC_DIR, "data", "prompts", "comparisons_template.md")
)

# Reemplazar variables
prompt_comparaciones = replace_variables(prompt_comparaciones, {
    "{marca}": marca,
    "{modelo}": modelo,
    "{pais}": pais,
    "{modelos_comparados}": modelos_comparados_texto.strip()
})

### Llamada a la API

In [15]:
# Llamada a la API
interaction_con_2 = client.interactions.create(
    model='gemini-3-pro-preview',
    input=prompt_comparaciones,
    tools=[{'type': 'google_search'}]  # Habilita Google Search
)

### Output de la API

In [16]:
final_output_comparaciones = ""
for output in interaction_con_2.outputs:
    if hasattr(output, "text"):
        final_output_comparaciones += output.text

### Output finales

In [17]:
print(final_output)

**DEEP SENTIMENT RESEARCH: Hero Hunk 125 R (Modelo 2026)**
**País:** Colombia

**0. Segmento y tipo de moto**
*   **Tipo según BD:** ['Urbana']
*   **Segmento identificado por usuarios:** Sport de entrada / Naked 125cc (Percibida como una moto de mayor cilindraje por su estética).

**1. Sentimiento general**
En Colombia, la Hunk 125 R es percibida como la moto de "apariencia de 160cc a precio de 125cc". El sentimiento predominante es de satisfacción visual y económica; los usuarios sienten que compraron "mucha moto" por su dinero, destacando su robustez frente a rivales más delgadas. Sin embargo, existe una frustración latente por su tecnología mecánica "anticuada" (carburador) frente a su rival directa, la TVS Raider (que ya ofrece inyección). Es amada por su estética agresiva pero criticada por detalles de refinamiento.

**2. Sensaciones de manejo más mencionadas**
*   **Estabilidad:** Muy elogiada. La llanta trasera ancha (120/80) y el peso del conjunto (133kg aprox.) hacen que se s

In [18]:
print(final_output_comparaciones)

Aquí tienes el análisis comparativo detallado de la **Hero Hunk 125 R** frente a sus competidores directos en Colombia, basado en experiencias de usuarios y datos de mercado.

### 6. Comparaciones naturales

#### **1. Hero Hunk 125 R vs. TVS Raider 125**
*   **Por qué los usuarios las comparan:**
    Es la rivalidad más fuerte del segmento en Colombia ("La Batalla de las Indias"). Los usuarios las cruzan porque buscan la **mejor moto de entrada** (primera moto) que no parezca básica. Ambas prometen estética deportiva premium a precio de 125cc. La duda recurrente en foros es: *"¿Me voy por la tecnología de la Raider o por la robustez de la Hunk?"*.
*   **En qué gana el modelo principal (Hero Hunk 125 R):**
    *   **Presencia física:** Los usuarios coinciden en que la Hunk 125 R se ve "más moto". Gana por sus llantas más anchas (130 atrás vs. 100 de la Raider) y un tanque más musculoso, lo que evita que se vea "delgada" o pequeña como critican a veces a la Raider.
    *   **Sencillez me

In [19]:
def merge_research_simple(final_output, final_output_comparaciones):
    """
    Une los outputs buscando el punto 7 para insertar el 6 antes
    """
    # Buscar "7. " en el texto
    if "7. " in final_output:
        # Dividir en la primera ocurrencia de "7. "
        before_7, from_7_onwards = final_output.split("7. ", 1)

        # Unir: antes del 7 + punto 6 + desde el 7 en adelante
        merged = before_7 + final_output_comparaciones + "\n\n7. " + from_7_onwards
        return merged
    else:
        # Si no encuentra el punto 7, agregar al final
        return final_output + "\n\n" + final_output_comparaciones

In [20]:
final_research_complete = merge_research_simple(final_output, final_output_comparaciones)


In [22]:
print(final_research_complete)

**DEEP SENTIMENT RESEARCH: Hero Hunk 125 R (Modelo 2026)**
**País:** Colombia

**0. Segmento y tipo de moto**
*   **Tipo según BD:** ['Urbana']
*   **Segmento identificado por usuarios:** Sport de entrada / Naked 125cc (Percibida como una moto de mayor cilindraje por su estética).

**1. Sentimiento general**
En Colombia, la Hunk 125 R es percibida como la moto de "apariencia de 160cc a precio de 125cc". El sentimiento predominante es de satisfacción visual y económica; los usuarios sienten que compraron "mucha moto" por su dinero, destacando su robustez frente a rivales más delgadas. Sin embargo, existe una frustración latente por su tecnología mecánica "anticuada" (carburador) frente a su rival directa, la TVS Raider (que ya ofrece inyección). Es amada por su estética agresiva pero criticada por detalles de refinamiento.

**2. Sensaciones de manejo más mencionadas**
*   **Estabilidad:** Muy elogiada. La llanta trasera ancha (120/80) y el peso del conjunto (133kg aprox.) hacen que se s

In [52]:
texto_datos_basicos_modelos_comparados = ""

for modelo in modelos_comparados:
    df_final_comparacion = df_columnas_finales[(df_columnas_finales["Marca"] == modelo["marca"]) & (df_columnas_finales["Modelo"] == modelo["modelo"])]
    for index, row in df_final_comparacion.iterrows():
        marca = row["Marca"]
        modelo = row["Modelo"]
        año = row["Año"]
        pais = pais
        tipo = row["Tipo"]
        code = row["code"]
        url = row["url"]
        texto_datos_basicos_modelos_comparados += f"""
Marca: {marca}
Modelo: {modelo}
Año: {año}
Pais: {pais}
Tipo: {tipo}
URL: {url}
        """

In [53]:
df_modelo_seleccionado_base_inventario = df_columnas_finales[df_columnas_finales["code"] == "CO2961-hero-hunk-125-r"]
for index, row in df_modelo_seleccionado_base_inventario.iterrows():
    marca = row["Marca"]
    modelo = row["Modelo"]
    año = row["Año"]
    pais = pais
    tipo = row["Tipo"]
    code = row["code"]
    url = row["url"]

dict_modelo_seleccionado_base_inventario = {
    "marca": marca,
    "modelo": modelo,
    "año": año,
    "pais": pais,
    "tipo": tipo,
}

In [54]:
#
# TODO: Agregar imágenes de modelo
#
datos_basicos =  f"""
Datos básicos del modelo:
Marca: {marca}
Modelo: {modelo}
Año: {año}
Pais: {pais}
Tipo: {tipo}
URL: {url}
"""

otros_datos = f"""
Datos de los demás modelos comparados:
{texto_datos_basicos_modelos_comparados}
"""

In [55]:
print(datos_basicos)


Datos básicos del modelo:
Marca: Hero
Modelo: Hunk 125 R
Año: 2026
Pais: Colombia
Tipo: ['Urbana']
URL: https://www.galgo.com/co/motos/CO2961-hero-hunk-125-r



In [56]:
print(otros_datos)


Datos de los demás modelos comparados:

Marca: TVS
Modelo: Raider 125
Año: 2026
Pais: Colombia
Tipo: ['Urbana']
URL: https://www.galgo.com/co/motos/CO731-tvs-raider-125
        
Marca: TVS
Modelo: Raider FI 125
Año: 2026
Pais: Colombia
Tipo: ['Urbana']
URL: https://www.galgo.com/co/motos/CO2861-tvs-raider-fi-125
        
Marca: Hero
Modelo: Hunk 150 XT
Año: 2026
Pais: Colombia
Tipo: ['Urbana', 'Naked']
URL: https://www.galgo.com/co/motos/CO2615-hero-hunk-150-xt
        



In [57]:
final_research_complete += datos_basicos + "\n\n" + otros_datos

In [58]:
print(final_research_complete)

**DEEP SENTIMENT RESEARCH: Hero Hunk 125 R (Modelo 2026)**
**País:** Colombia

**0. Segmento y tipo de moto**
*   **Tipo según BD:** ['Urbana']
*   **Segmento identificado por usuarios:** Sport de entrada / Naked 125cc (Percibida como una moto de mayor cilindraje por su estética).

**1. Sentimiento general**
En Colombia, la Hunk 125 R es percibida como la moto de "apariencia de 160cc a precio de 125cc". El sentimiento predominante es de satisfacción visual y económica; los usuarios sienten que compraron "mucha moto" por su dinero, destacando su robustez frente a rivales más delgadas. Sin embargo, existe una frustración latente por su tecnología mecánica "anticuada" (carburador) frente a su rival directa, la TVS Raider (que ya ofrece inyección). Es amada por su estética agresiva pero criticada por detalles de refinamiento.

**2. Sensaciones de manejo más mencionadas**
*   **Estabilidad:** Muy elogiada. La llanta trasera ancha (120/80) y el peso del conjunto (133kg aprox.) hacen que se s

### Exportar OUTPUT

In [ ]:
with open(f"{SRC_DIR}/data/output_gemini/{code}-{pais}.md", "w", encoding="utf-8") as f:
    f.write(final_output)

### Usando Replicate

In [ ]:
# client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

# output = client.run(
#     ref = "google/gemini-3-pro",
#     input={
#         "prompt": prompt
#     },
# )

# print("\n--- RESPUESTA ---\n")
# if isinstance(output, list):
#     print("".join([str(x) for x in output]))
# else:
#     print(output)